<a href="https://colab.research.google.com/github/woojoo123/Project/blob/main/Restaurant_sentiment_analysis/Crawling.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install selenium
!apt-get update
!apt install chromium-chromedriver
!cp /usr/lib/chromium-browser/chromedriver /usr/bin
!pip install chromedriver-autoinstaller

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.7/9.7 MB 45.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 481.7/481.7 kB 20.2 MB/s eta 0:00:00
Get:1 http://security.ubuntu.com/ubuntu jammy-security InRelease [129 kB]
Get:2 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,626 B]
Get:3 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease [1,581 B]
Get:4 https://r2u.stat.illinois.edu/ubuntu jammy InRelease [6,555 B]
Hit:5 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:6 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [128 kB]
Get:7 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  Packages [1,196 kB]
Get:8 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease [18.1 kB]
Get:9 http://security.ubuntu.com/ubuntu jammy-security/restricted amd64 Packages [3,436 kB]
Get:10 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease [24.3 kB]
Get:11 htt

In [ ]:
import selenium
print(selenium.__version__)

4.27.1


In [ ]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
import chromedriver_autoinstaller
import requests
from bs4 import BeautifulSoup
import time
from time import sleep
import requests
import re
import os

# ChromeDriver 자동 설치
chromedriver_autoinstaller.install()

# Selenium WebDriver 설정
options = webdriver.ChromeOptions()
options.add_argument('--headless')    # GUI 없이 실행
options.add_argument('--no-sandbox')    # 보안 설정 비활성화
options.add_argument('--disable-dev-shm-usage')    # 메모리 사용 최적화

# WebDriver 초기화
driver = webdriver.Chrome(options=options)

# 테스트: Kakao.map 홈페이지 접속
driver.get("https://map.kakao.com/")
print(driver.title)  # Google 홈페이지 제목 출력

# WebDriver 종료
driver.quit()

카카오맵


In [ ]:
driver = webdriver.Chrome(options=options)
driver.get('https://map.kakao.com/')

# 검색창에 '강남역 맛집' 입력
search = driver.find_element(By.ID, "search.keyword.query")
search.send_keys('강남역 맛집')

# 가려져 있는 요소를 삭제
driver.execute_script("document.getElementById('dimmedLayer').remove();")

search_Btn = driver.find_element(By.ID, 'search.keyword.submit')
search_Btn.click()
time.sleep(2)

# 장소 더보기
btnMore = driver.find_element(By.ID, 'info.search.place.more')
btnMore.click()

# 페이지 번호데이터
page = driver.find_elements(By.CSS_SELECTOR, '#info\.search\.page > div > a')

In [ ]:
page_urls = []

try:
    while True:
        for i in range(6):
            if i < 5:
                # 5페이지까지 데이터 수집
                page[i].click()
                time.sleep(2)
                html = driver.page_source # 현재 브라우저의 소스
                soup = BeautifulSoup(html, 'html.parser')
                moreviews = soup.find_all(name='a', attrs={'class':'moreview'}) # 상세보기
                for moreview in moreviews:
                    page_url = moreview.get('href')
                    page_urls.append(page_url)
            else:
                more = driver.find_element(By.ID, 'info.search.page.next')
                more.click()
except:
    print('수집 완료!')

driver.close()
print(page_urls)
print(len(page_urls))

In [ ]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from bs4 import BeautifulSoup
import pandas as pd
import time

# Selenium WebDriver 초기화
driver = webdriver.Chrome()

# 결과 저장용 리스트
stars = []
reviews = []

for page in page_urls:
    print(f"크롤링 시작: {page}")
    driver.get(page)  # 페이지 로드
    time.sleep(2)  # 페이지 로딩 대기

    # "후기 더보기" 버튼 반복 클릭
    while True:
        try:
            # "후기 더보기" 버튼 찾기
            more_buttons = driver.find_elements(By.CSS_SELECTOR, "a.link_more > span.txt_more")
            clicked = False  # 버튼 클릭 여부 확인

            for button in more_buttons:
                if "후기 더보기" in button.text:
                    button.click()
                    time.sleep(2)  # 데이터 로드 대기
                    clicked = True
                    break

            # 더 이상 클릭할 "후기 더보기" 버튼이 없으면 루프 종료
            if not clicked:
                print("더 이상 '후기 더보기' 버튼이 없습니다.")
                break

        except Exception as e:
            print(f"오류 발생: {e}")
            break

    # 모든 "후기 더보기" 버튼 클릭 후 HTML 데이터 수집
    html = driver.page_source
    soup = BeautifulSoup(html, "html.parser")

    # 리뷰와 별점이 포함된 블록 찾기
    review_blocks = soup.select("ul.list_evaluation > li")

    for block in review_blocks:
        try:
            # 별점 추출 (리뷰 블록 내에서만 찾기)
            star_element = block.select_one("div.star_info > div.grade_star.size_s > span.ico_star.star_rate > span.ico_star.inner_star")

            if star_element:
                style = star_element.get("style", "")
                if "width" in style:
                    width_percentage = int(style.split(":")[1].replace("%", "").replace(";", "").strip())
                    star_rating = (width_percentage / 100) * 5  # 별점 변환
                    stars.append(star_rating)
                else:
                    stars.append(None)
            else:
                stars.append(None)

            # 리뷰 추출 (리뷰 블록 내에서만 찾기)
            review_element = block.select_one("div.comment_info > p.txt_comment > span")
            if review_element:
                review_text = review_element.get_text(separator="\n", strip=True)
                reviews.append(review_text)
            else:
                reviews.append(None)

        except Exception as e:
            print(f"데이터 추출 오류: {e}")
            stars.append(None)
            reviews.append(None)

# WebDriver 종료
driver.quit()

# DataFrame에 저장
df = pd.DataFrame({'review': reviews, 'score': stars})
print(df.head())

# CSV 파일 저장
df.to_csv("kakaomap_reviews.csv", index=False, encoding="utf-8")
print("CSV 파일 저장 완료!")
